# Installs

In [1]:
! pip install langchain

# Load PDFs.
! pip install pypdf

# Access to the Google Palm API
! pip install google-generativeai

# Database for embeddings
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.0/255.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 11.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━

# Imports and setup

In [53]:
import os

In [2]:
# Connect to drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Palm Setup

In [5]:
# Generate yours @ https://developers.generativeai.google/products/palm
with open('/content/drive/My Drive/key.txt', 'r') as f:
  palm_api_key = f.readlines()[0].strip('\n')

os.environ["GOOGLE_API_KEY"] = palm_api_key

In [6]:
import langchain
from langchain.prompts import PromptTemplate
from langchain.llms.google_palm import GooglePalm
from langchain.embeddings.google_palm import GooglePalmEmbeddings

llm = GooglePalm(temperature=0.0, max_output_tokens=256)
embeddings = GooglePalmEmbeddings()

# Loading Documents

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [10]:
doc_path = "https://www.abc.xyz/assets/86/99/68122c444c4a93d2228e21ecc16b/20230426-alphabet-10q.pdf"
loader = PyPDFLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Vectorstoe and Retrievers

In [18]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [19]:
docsearch = Chroma.from_documents(texts, embeddings, collection_name="google_2023")

In [20]:
google_23_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

# Memory

In [24]:
from langchain.memory import ConversationBufferMemory

In [25]:
memory = ConversationBufferMemory(memory_key="chat_history")

# Creating Agent

In [30]:
from langchain import LLMChain

from langchain.agents import load_tools, AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import AgentType

In [47]:
# pre-built
calc_tool = load_tools(["llm-math"], llm=llm)[0]
# custom
retreival_tool = Tool(
    name="Google Financial Info QA System",
    func=google_23_qa.run,
    description="useful for when you need to answer questions about any financial information related to Google",
)
tools = [retreival_tool, calc_tool]

In [48]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [49]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent_with_memory = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_executor_with_memory = AgentExecutor.from_agent_and_tools(
    agent=agent_with_memory, tools=tools, verbose=True, memory=memory
)

In [50]:
agent_executor_with_memory.run("What was Google's total income from operations in 2022?")



> Entering new AgentExecutor chain...
Thought: I need to use Google Financial Info QA System to get the answer to this question
Action: Google Financial Info QA System
Action Input: total income from operations in 2022
Observation: 20,094
Thought:I now know the final answer
Final Answer: 20,094

> Finished chain.


'20,094'

In [51]:
agent_executor_with_memory.run("What about 2023?")



> Entering new AgentExecutor chain...
Thought: I need to use Google Financial Info QA System to get Google's total income from operations in 2023
Action: Google Financial Info QA System
Action Input: total income from operations in 2023
Observation: 17,415
Thought:I now know the final answer
Final Answer: 17,415

> Finished chain.


'17,415'

In [52]:
agent_executor_with_memory.run("Different between these raised to power 4?")



> Entering new AgentExecutor chain...
Thought: I need to find the difference between Google's total income from operations in 2022 and 2023, then raise it to the power of 4
Action: Calculator
Action Input: 20094 - 17415
Observation: Answer: 2679
Thought:I now know the difference between Google's total income from operations in 2022 and 2023
Action: Calculator
Action Input: 2679^4
Observation: Answer: 51509917515681
Thought:I now know the final answer
Final Answer: 51509917515681

> Finished chain.


'51509917515681'